#### Re-run KNN, VT, CTF, CSF only for those appeared in AMT labeled data
- Re-run only for word pairs appear in AMT labeled file <br>
- KNN, VT, CTF, CSF\_200, CSF\_2000, CSF with new parameters, weighted\_KNN, combo method <br>
- Combine new treatment effects into AMT labeled file after re-run

In [127]:
import pickle,time,ast,csv
import pandas as pd
from nltk import pos_tag
import warnings
warnings.filterwarnings("ignore")

AMT_path = '/data/2/zwang/2018_S_WordTreatment/V2_AMT/'

In [65]:
def locate_AMT_sents(amt_pd,data_pd):
    """
    Locate the sentence id from original data file for each AMT labeled sentence.
    """
    sentid_list = []
    for idx,row in amt_pd.iterrows():
        wdpair_pd = data_pd[(data_pd['source'] == row['source']) & (data_pd['target'] == row['target'])]
        pair_sents = wdpair_pd['source_pos_sents'].values[0] + wdpair_pd['source_neg_sents'].values[0]
        try:
            sentid_list.append(pair_sents.index(row['src_sentence']))
        except:
#             return row,wdpair_pd
            sentid_list.append(-1)
    return sentid_list

In [119]:
def get_effect(effect_pd,amt_pd,method):
    effect_list = []
    for idx,row in amt_pd.iterrows():
        wdpair_pd = effect_pd[(effect_pd['source'] == row['source']) & (effect_pd['target'] == row['target'])]
        if(method != 'combo_effect'):
            pair_sents = ast.literal_eval(wdpair_pd['source_pos_sents_treatment'].values[0])[0] + ast.literal_eval(wdpair_pd['source_neg_sents_treatment'].values[0])[0]
        else:
            pair_sents = ast.literal_eval(wdpair_pd['source_pos_context_prob'].values[0])[0] + ast.literal_eval(wdpair_pd['source_neg_context_prob'].values[0])[0]
        effect_list.append(pair_sents[row['locate_id']][1])
    return effect_list

In [125]:
def get_wd_ratio(combo_pd,amt_pd):
    src_pos_ratio = []
    tar_pos_ratio = []
    for idx,row in amt_pd.iterrows():
        wdpair_pd = combo_pd[(combo_pd['source'] == row['source']) & (combo_pd['target'] == row['target'])]
        src_pos_ratio.append(wdpair_pd['src_pos_ratio'].values[0])
        tar_pos_ratio.append(wdpair_pd['tar_pos_ratio'].values[0])
    
    return src_pos_ratio,tar_pos_ratio

In [126]:
def combine_all_effects(AMT_file,data_file,result_file,effect_filelist):
    amt_pd = pd.read_csv(AMT_file,index_col=False)
    data_pd = pd.DataFrame(pickle.load(open(data_file,'rb')))
#     row,wdpair_pd = locate_AMT_sents(amt_pd,data_pd)
#     return row,wdpair_pd
    amt_pd['locate_id'] = locate_AMT_sents(amt_pd,data_pd)
    correct_pd = amt_pd[amt_pd['locate_id'] != -1]
    init_cols = list(amt_pd.columns)
#     return amt_pd
    
    for method in ['KNN','VT','CTF','CSF_200','CSF_2000','CSF_param','weighted_KNN','combo_effect']:
        effect_pd = pd.read_csv(effect_filelist[method],index_col=False)
#         return effect_pd
        correct_pd[method] = get_effect(effect_pd,correct_pd,method)
    
    combo_pd = pd.read_csv(effect_filelist['combo_effect'],index_col=False)
    src_pos_ratio,tar_pos_ratio = get_wd_ratio(combo_pd,correct_pd)
    correct_pd['src_pos_ratio'] = src_pos_ratio
    correct_pd['tar_pos_ratio'] = tar_pos_ratio

    correct_pd.to_csv(result_file,columns = init_cols+['KNN','VT','CTF','CSF_200','CSF_2000','CSF_param','weighted_KNN','combo_effect']+['src_pos_ratio','tar_pos_ratio'],
                  index = False)
    return correct_pd

In [132]:
prefix = 'tw'
effect_filemap = {}
effect_filemap['KNN'] = AMT_path+'AMT_WdSents/3_KNN/'+prefix+'_knn_30_treatment.csv'
effect_filemap['VT'] = AMT_path+'AMT_WdSents/3_VT/'+prefix+'_vt_200tree_treatment.csv'
effect_filemap['CTF'] = AMT_path+'AMT_WdSents/3_CTF/'+prefix+'_ctf_200tree_treatment.csv'
effect_filemap['CSF_200'] = AMT_path+'AMT_WdSents/3_CSF/'+prefix+'_csf_200tree_treatment.csv'
effect_filemap['CSF_2000'] = AMT_path+'AMT_WdSents/3_CSF/'+prefix+'_csf_2000tree_treatment.csv'
effect_filemap['CSF_param'] = AMT_path+'AMT_WdSents/3_CSF/'+prefix+'_csf_param_treatment.csv'
effect_filemap['weighted_KNN'] = AMT_path+'AMT_WdSents/3_KNN/'+prefix+'_weightedknn_30_treatment.csv'
effect_filemap['combo_effect'] = AMT_path+'AMT_WdSents/3_Combo/'+prefix+'_AMT_combo.csv'

start = time.time()
correct_pd = combine_all_effects(AMT_file = AMT_path+'6_amt/results/'+prefix+'_result_weightedAMT_nodummy.csv',
                             data_file = AMT_path+'AMT_WdSents/Data/'+prefix+'_AMT_wdsents.pickle',
                             result_file = AMT_path+'AMT_WdSents/Effects/'+prefix+'_AMT_comboEffects.csv',
                             effect_filelist = effect_filemap)
end = time.time()
print((end-start)/60)

0.10294694503148397


In [133]:
correct_pd.head()

,id,true_y,source,target,src_sentence,tar_sentence,knn_effect,vt_effect,ctf_effect,csf_effect,...,KNN,VT,CTF,CSF_200,CSF_2000,CSF_param,weighted_KNN,combo_effect,src_pos_ratio,tar_pos_ratio
0,1517508,0,cute,super,Cute cute & cute,Cute super & super,0.96667,0.08365,0.13391,0.12218,...,0.03333,0.04766,0.10387,0.07853,0.07706,0.07488,0.06348,0.29328,0.283086,0.361538
2,1515888,1,cute,super,Aubrey Plaza is so cute,Aubrey Plaza is so super,0.10000,0.07120,0.15098,0.13508,...,-0.06667,0.04439,0.12244,0.07461,0.08780,0.09107,0.05958,0.25567,0.283086,0.361538
5,2207163,0,gay,homo,"Why is port so gay man , like for real -.-","Why is port so homo man , like for real -.-",0.26667,0.07077,0.14826,0.17637,...,0.36667,0.05923,0.19665,0.18020,0.20607,0.28455,0.02961,0.60723,0.597101,0.792453
7,2371713,0,awkward,weird,That awkward moment when you pass out at work ...,That weird moment when you pass out at work #o...,-0.43333,-0.00844,-0.01591,0.01893,...,-0.46667,-0.01014,-0.07093,-0.03068,-0.00356,-0.05499,-0.15682,0.45925,0.298770,0.328742
8,2371241,1,awkward,weird,An this is when group chats get awkward,An this is when group chats get weird,0.00000,0.04468,0.06481,0.06103,...,0.03333,0.02196,0.02409,0.07540,0.07999,0.08277,0.02242,0.30559,0.298770,0.328742


In [91]:
error_pd = amt_pd[amt_pd['locate_id'] == -1]
error_pd.shape

(46, 18)

In [100]:
effect_pd = pd.read_csv(AMT_path+'AMT_WdSents/3_KNN/'+prefix+'_knn_30_treatment.csv',index_col=False)
effect_pd.shape

(38, 6)

In [108]:
wdpair_pd = effect_pd[(effect_pd['source'] == 'famous') & (effect_pd['target'] == 'wonderful')]

In [114]:
ast.literal_eval(wdpair_pd['source_pos_sents_treatment'].values[0])[0]

('Neighborhood on _NNP_ _NNP_ and steps away from the _NNP_ _NNP_ restaurants, shops, _NUMBER_ _NNP_ _NNP_ and the famous _NNP_ _NNP_ _NNP_ ',
 0.06667)

In [105]:
effect_pd.head()

,source,target,source_pos_sents_treatment,source_neg_sents_treatment,target_pos_sents_treatment,target_neg_sents_treatment
0,famous,wonderful,[[('Neighborhood on _NNP_ _NNP_ and steps away...,[[('_NNP_ just a quick _NNP_ ride to The _NNP_...,[[('Enjoy their wonderful food and cozy ambian...,"[[('A wonderful _NNP_ neighborhood. ', 0.03333..."
1,store,boutique,[[('There is also a Whole Foods grocery store ...,[[('The apartment is right next to a grocery s...,[[('Enjoy the restaurants and shopping of _NNP...,"[[(""Home to many of the city's finest restaura..."
2,famous,grand,[[('_NUMBER_ minutes bike ride to famous beaut...,[[('_NNP_ just a quick _NNP_ ride to The _NNP_...,[[('On _NNP_ the local grand lake area is home...,[[('Down _NUMBER_ block south is the grand _NN...
3,famous,excellent,[[('Neighborhood on _NNP_ _NNP_ and steps away...,[[('_NNP_ just a quick _NNP_ ride to The _NNP_...,[[('Less than half a block from _NNP_ _NNP_ wh...,[[('_NNP_ beautiful with all the brick brownst...
4,famous,big,[[('_NUMBER_ minutes bike ride to famous beaut...,[[('You can explore the stunning Southern styl...,[[('_NNP_ _NNP_ _NNP_ and its big city surroun...,"[[('There is also a tennis court, library and ..."


In [80]:
data_pd = pd.DataFrame(pickle.load(open(AMT_path+'AMT_WdSents/Data/'+prefix+'_AMT_wdsents.pickle','rb')))
wdpair_pd = data_pd[(data_pd['source'] == 'gay') & (data_pd['target'] == 'homo')]

In [81]:
print(error_pd.iloc[1]['src_sentence'])
print(pos_tag(error_pd.iloc[1]['src_sentence'].split()))

So obama was a homophobic teen who beat men he thot were gay . And if gay , then a rapist per Rachel #Zimmerman
[('So', 'RB'), ('obama', 'NN'), ('was', 'VBD'), ('a', 'DT'), ('homophobic', 'JJ'), ('teen', 'NN'), ('who', 'WP'), ('beat', 'VBD'), ('men', 'NNS'), ('he', 'PRP'), ('thot', 'VBZ'), ('were', 'VBD'), ('gay', 'NNS'), ('.', '.'), ('And', 'CC'), ('if', 'IN'), ('gay', 'NNS'), (',', ','), ('then', 'RB'), ('a', 'DT'), ('rapist', 'NN'), ('per', 'IN'), ('Rachel', 'NNP'), ('#Zimmerman', 'NN')]


In [85]:
print(pos_tag('So obama was a homophobic teen who beat men he thot were homo . And if gay , then a rapist per Rachel #Zimmerman'.split()))

[('So', 'RB'), ('obama', 'NN'), ('was', 'VBD'), ('a', 'DT'), ('homophobic', 'JJ'), ('teen', 'NN'), ('who', 'WP'), ('beat', 'VBD'), ('men', 'NNS'), ('he', 'PRP'), ('thot', 'VBZ'), ('were', 'VBD'), ('homo', 'NN'), ('.', '.'), ('And', 'CC'), ('if', 'IN'), ('gay', 'NNS'), (',', ','), ('then', 'RB'), ('a', 'DT'), ('rapist', 'NN'), ('per', 'IN'), ('Rachel', 'NNP'), ('#Zimmerman', 'NN')]


- Double check if mapped effects in each method is correct

In [161]:
prefix = 'airbnb'
effect_filemap = {}
effect_filemap['KNN'] = AMT_path+'AMT_WdSents/3_KNN/'+prefix+'_knn_30_treatment.csv'
effect_filemap['VT'] = AMT_path+'AMT_WdSents/3_VT/'+prefix+'_vt_200tree_treatment.csv'
effect_filemap['CTF'] = AMT_path+'AMT_WdSents/3_CTF/'+prefix+'_ctf_200tree_treatment.csv'
effect_filemap['CSF_200'] = AMT_path+'AMT_WdSents/3_CSF/'+prefix+'_csf_200tree_treatment.csv'
effect_filemap['CSF_2000'] = AMT_path+'AMT_WdSents/3_CSF/'+prefix+'_csf_2000tree_treatment.csv'
effect_filemap['CSF_param'] = AMT_path+'AMT_WdSents/3_CSF/'+prefix+'_csf_param_treatment.csv'
effect_filemap['weighted_KNN'] = AMT_path+'AMT_WdSents/3_KNN/'+prefix+'_weightedknn_30_treatment.csv'
effect_filemap['combo_effect'] = AMT_path+'AMT_WdSents/3_Combo/'+prefix+'_AMT_combo.csv'

In [164]:
src_wd = 'yummy'
tar_wd = 'delicious'
locate_id = 16
for md in effect_filemap:
    md_pd = pd.read_csv(effect_filemap[md],index_col=False)
    wdpair_pd = md_pd[(md_pd['source'] == src_wd) & (md_pd['target'] == tar_wd)]
    if(md != 'combo_effect'):
        all_sents = ast.literal_eval(wdpair_pd['source_pos_sents_treatment'].values[0])[0] + ast.literal_eval(wdpair_pd['source_neg_sents_treatment'].values[0])[0]
        print(md,all_sents[locate_id][1])
    elif(md == 'combo_effect'):
        all_sents = ast.literal_eval(wdpair_pd['source_pos_context_prob'].values[0])[0] + ast.literal_eval(wdpair_pd['source_neg_context_prob'].values[0])[0]
        print(md,all_sents[locate_id][1],wdpair_pd['src_pos_ratio'].values[0],wdpair_pd['tar_pos_ratio'].values[0])

VT -0.00251
CSF_200 -0.06901
CSF_param -0.11022
CSF_2000 -0.06425
KNN 0.06667
weighted_KNN 0.08001
CTF 0.04213
combo_effect 0.82622 0.805555555556 0.831417624521


In [165]:
wdpair_pd

,source,target,src_pos_ratio,tar_pos_ratio,source_pos_context_prob,source_neg_context_prob,target_pos_context_prob,target_neg_context_prob
33,yummy,delicious,0.805556,0.831418,[[('There are plenty of incredible restaurants...,"[[('For grilling, you probably want to go to _...","[[(""Close by, we love Canseco's _NNP_ _NNP_ _N...",[[('throughout _NNP_ _NNP_ and find hundreds o...
